In [1]:
import get_api_data,git_local
import pandas as pd
from flatten_dict import flatten
from datetime import datetime
from dateutil import relativedelta
import random as rd
import os

ImportError: attempted relative import with no known parent package

In [2]:
access_token = 'ef66b5bf5fd6c14d63586bf00caa9e72c78cc675'
repo_owner = 'rspec'
git_url = 'git://github.com/rspec/rspec-rails.git'
api_base_url = 'https://api.github.com'
repo_name = 'rspec-rails'
client = get_api_data.get_data(access_token,repo_owner,api_base_url,repo_name) 

In [3]:
#client.get_users()

In [4]:
local = git_local.git_local(git_url,repo_name)

In [5]:
local.clone_repo()

pygit2.Repository('/Users/suvodeepmajumder/Documents/Class/CSC510/csc510-project/code_base/git_access/temp_repo/rspec-rails/.git/')

In [7]:
commits = local.get_commits()
commit_df = pd.DataFrame(commits, columns = ['commit_id','commit_time','author','author_email',
                                             'committer','committer_email','comments','commit_parent','buggy'])

In [8]:
committed_files = local.get_committed_files()

In [9]:
diffs = local.get_diffs(commit_df.commit_id.values.tolist())

In [11]:
committed_files_df = pd.DataFrame(committed_files, columns = ['commit_id','file_id','modification_type','file_name','buggy'])

In [12]:
latest_commit_time = {}
oldest_commit_time = {}
for commit in commits:
    if commit[0] == 'ea6bdef57db2906323deeaa648cabab69ebbdefc':
        continue
    try:
        for file_id in diffs[commit[0]]['files'].keys():
            if diffs[commit[0]]['files'][file_id]['file_path'] not in latest_commit_time:
                latest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] = commit[1]
                oldest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] = commit[1]
            else:
                if latest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] < commit[1]:
                    latest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] = commit[1]
                if oldest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] > commit[1]:
                    oldest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']] = commit[1]
    except ValueError:
        print(commit[0])
        continue

In [13]:
uc1 = []
for commit in commits:
    try:
        if commit[0] == 'ea6bdef57db2906323deeaa648cabab69ebbdefc':
            continue
        for file_id in diffs[commit[0]]['files'].keys():
            file = os.getcwd() + '/temp_repo/' + repo_name + '/' + diffs[commit[0]]['files'][file_id]['file_path']
            exists = os.path.isfile(file)
            if exists:
                # Store configuration file values
                latest = datetime.strptime(latest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']],'%Y-%m-%d %H:%M:%S')
                oldest = datetime.strptime(oldest_commit_time[diffs[commit[0]]['files'][file_id]['file_path']],'%Y-%m-%d %H:%M:%S')
                current = datetime.strptime(commit[1],'%Y-%m-%d %H:%M:%S')
                # Normalization based on ration between delta of oldest chnage and latest change
                latest_diff = abs(relativedelta.relativedelta(current,latest).months)
                oldest_diff = abs(relativedelta.relativedelta(current,oldest).months)
                total_diff = abs(relativedelta.relativedelta(latest,oldest).months)
                if latest_diff == 0:
                    latest_diff = 1
                if oldest_diff == 0:
                    oldest_diff = 1  
                if total_diff == 0:
                    total_diff = 1
                time_norm = (oldest_diff/latest_diff)/total_diff

                uc1.append([commit[0],commit[1],commit[2],commit[3],
                           diffs[commit[0]]['files'][file_id]['file_path'],
                            diffs[commit[0]]['files'][file_id]['new_lines']*time_norm])
            else:
                # Keep presets
                continue
    except KeyError:
        print(commit[0])
        continue

In [14]:
uc1_df = pd.DataFrame(uc1,columns = ['commit_id','commit_time','AuthorName','AuthorEmail','FilePath',
                                     'Score'])

In [15]:
uc1_df_matrix = uc1_df.drop(labels = ['commit_time','commit_id'],axis = 1)

In [16]:
uc1_df_dev_sum = uc1_df_matrix.groupby(['FilePath','AuthorEmail','AuthorName'],as_index=False).sum()
uc1_df_dev_sum.sort_values(by=['FilePath','Score'],inplace = True,ascending = False)

In [61]:
uc1_df_dev_sum['LinesOfCode'] = [0]*uc1_df_dev_sum.shape[0]
uc1_df_dev_sum['FileSizeBytes'] = [0]*uc1_df_dev_sum.shape[0]
uc1_df_dev_sum['LastModified'] = [0]*uc1_df_dev_sum.shape[0]
uc1_df_dev_sum['RepoName'] = [0]*uc1_df_dev_sum.shape[0]
uc1_df_dev_sum['RepoUrl'] = [0]*uc1_df_dev_sum.shape[0]
for i in range(uc1_df_dev_sum.shape[0]):
    try:
        file = os.getcwd() + '/temp_repo/' + repo_name + '/' + uc1_df_dev_sum.iloc[i,0]
        file_size = os.path.getsize(file)
        num_lines = sum(1 for line in open(file))
    except:
        file_size = 0
        num_lines = 0
        print("Not there")
        continue
    uc1_df_dev_sum.iloc[i,4] = num_lines
    uc1_df_dev_sum.iloc[i,5] = file_size
    uc1_df_dev_sum.iloc[i,6] = latest_commit_time[uc1_df_dev_sum.iloc[i,0]]
    uc1_df_dev_sum.iloc[i,7] = repo_name
    uc1_df_dev_sum.iloc[i,8] = git_url

In [62]:
import random
from datetime import datetime, timedelta

min_year=1900
max_year=datetime.now().year

start = datetime(min_year, 1, 1, 00, 00, 00)
years = max_year - min_year+1
end = start + timedelta(days=365 * years)

for i in range(10):
    random_date = start + (end - start) * random.random()
    print(random_date)

#done

# or a function
def gen_datetime(min_year=1900, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

1904-08-15 16:50:46.177639
1988-06-17 21:30:45.727008
1992-12-05 10:58:36.522581
1912-06-30 01:25:19.191494
1913-07-22 15:45:16.253043
1905-09-09 07:44:28.801623
1915-12-15 09:23:24.715475
1995-06-23 11:15:23.254972
1981-01-22 03:58:58.083380
1926-01-22 07:06:38.820049


In [100]:
uc1_df_dev_sum.to_csv('uc1.csv')

In [63]:
uc1_df_dev_sum[uc1_df_dev_sum.FilePath == 'lib/rspec/rails/example/request_example_group.rb']

,FilePath,AuthorEmail,AuthorName,Score,LinesOfCode,FileSizeBytes,LastModified,RepoName,RepoUrl
1048,lib/rspec/rails/example/request_example_group.rb,dchelimsky@gmail.com,David Chelimsky,285.118874,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1046,lib/rspec/rails/example/request_example_group.rb,dan@agencyfusion.com,Dan Rasband,51.200000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1053,lib/rspec/rails/example/request_example_group.rb,jko170@gmail.com,Justin Ko,24.000000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1055,lib/rspec/rails/example/request_example_group.rb,mail@jonrowe.co.uk,Jon Rowe,22.950000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1056,lib/rspec/rails/example/request_example_group.rb,myron.marston@gmail.com,Myron Marston,15.800000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1052,lib/rspec/rails/example/request_example_group.rb,iybetesh@gmail.com,Isaac Betesh,11.000000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1051,lib/rspec/rails/example/request_example_group.rb,hello@xaviershay.com,Xavier Shay,9.600000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1057,lib/rspec/rails/example/request_example_group.rb,xavier@rhnh.net,Xavier Shay,9.600000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1050,lib/rspec/rails/example/request_example_group.rb,hello@jonrowe.co.uk,Jon Rowe,5.822727,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
1043,lib/rspec/rails/example/request_example_group.rb,aaron.kromer@gmail.com,Aaron Kromer,4.472727,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git


In [33]:
uc1_df

,commit_id,commit_time,AuthorName,AuthorEmail,FilePath,Score,LinesOfCode,FileSizeBytes,LastModified,RepoName,RepoUrl
0,15e1656a537bd2b2ce49a294870c68e3707391d6,2010-01-08 00:26:14,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/controller_example_gro...,1.200000,216,7369,2016-06-14 07:16:25,rspec-rails,git://github.com/rspec/rspec-rails.git
1,eba85a35ff08cf88bc9e6665c1a46c4b5dc4864f,2010-01-08 08:27:59,David Chelimsky,dchelimsky@gmail.com,.gitignore,0.027778,24,238,2015-07-10 01:11:30,rspec-rails,git://github.com/rspec/rspec-rails.git
2,eba85a35ff08cf88bc9e6665c1a46c4b5dc4864f,2010-01-08 08:27:59,David Chelimsky,dchelimsky@gmail.com,Rakefile,0.040000,224,6456,2017-11-15 17:40:25,rspec-rails,git://github.com/rspec/rspec-rails.git
3,a2e61070b4f432889750b9296d231e39dbe57d2f,2010-01-08 11:27:21,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/request_example_group.rb,1.720000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
4,857a0b3b14e59cb68a530e6a9815d59316f4be6b,2010-01-08 15:11:23,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/controller_example_gro...,1.280000,216,7369,2016-06-14 07:16:25,rspec-rails,git://github.com/rspec/rspec-rails.git
5,857a0b3b14e59cb68a530e6a9815d59316f4be6b,2010-01-08 15:11:23,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/request_example_group.rb,1.360000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git
6,d85243d328670b70de1f40f4a52a23d60f596e0b,2010-01-08 16:20:11,David Chelimsky,dchelimsky@gmail.com,.gitignore,0.055556,24,238,2015-07-10 01:11:30,rspec-rails,git://github.com/rspec/rspec-rails.git
7,c487e51095a66a517ef698c5d23fa04a7f042c92,2010-01-08 16:20:25,David Chelimsky,dchelimsky@gmail.com,Rakefile,0.180000,224,6456,2017-11-15 17:40:25,rspec-rails,git://github.com/rspec/rspec-rails.git
8,f06aa507280f8f9e6316c9bd5f72e640269a49b1,2010-01-08 17:53:26,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/controller_example_gro...,1.600000,216,7369,2016-06-14 07:16:25,rspec-rails,git://github.com/rspec/rspec-rails.git
9,f06aa507280f8f9e6316c9bd5f72e640269a49b1,2010-01-08 17:53:26,David Chelimsky,dchelimsky@gmail.com,lib/rspec/rails/example/request_example_group.rb,1.320000,30,786,2016-07-07 11:45:27,rspec-rails,git://github.com/rspec/rspec-rails.git


In [20]:
uc1_df_matrix.groupby(['FilePath'],set_index = )

TypeError: group() got an unexpected keyword argument 'set_index'